In [ ]:
# Methods for creating / working with tree-shaped networks to demonstrate / benchmark
# functionality for PixelArrayPowerFlow
push!(LOAD_PATH, "../jlfiles/")

using PixelArrayPowerFlow: Network, Branch, Bus, Gen, SlackParams, PQParams
using PixelArrayPowerFlow

using Plots

In [ ]:
# Creates a sample binary tree network with given height and 2^(height + 1) - 1 nodes
# The very top node is a slack bus, the rest are all generic PQ buses
function createbinarytreenetwork(height::Int)
    if height < 0
        error("Height must be a nonnegative integer.")
    end
    
    # Create empty arrays of branches, buses, gens
    branches = Array{Branch}(undef, 2^(height+1) - 2)
    buses = Array{Bus}(undef, 2^(height+1) - 1)
    gens = Array{Gen}(undef, 1)

    # Create a head node for the network (slack bus with a generator attached)
    (Pmin, Pmax) = (-1.0, 1.0)
    (Qmin, Qmax) = (-1.0, 1.0)
    (Vmin, Vmax) = (-1.0, 1.0) # this doesn't actually matter to the problem
    (θmin, θmax) = (-π/6, π/6) # this doesn't actually matter to the problem
    (Bshunt, Gshunt) = (0.0, 0.0)
    (V, θ) = (1.0, 0.0)
    id = 1
    gen = Gen(Pmin, Pmax, Qmin, Qmax)
    slackparams = SlackParams(V, θ)
    head = Bus(id, Vmin, Vmax, θmin, θmax, Bshunt, Gshunt, gen, slackparams)

    # Add head node, generator to array
    gens[1] = gen
    buses[1] = head

    # Starting from the very top, use recursion to create other buses
    # (and branches linking the other buses to the nodes directly above)
    # and populate branch / bus arrays with these
    binarytreeaux!(height, head, branches, buses)

    # Combine these into a network structure
    basemva = 100.0
    network = Network(branches, buses, gens, basemva)

    return network
end

# Recursive auxiliary function for createbinarytreenetwork. If curr is not a leaf then
# create children for curr and branches connecting curr to children, adding both to arrays.
# Then continue for each child with a reduced height.
# (branches and buses are the only arguments modified in the process)
function binarytreeaux!(height::Int, curr::Bus, branches::Array{Branch,1}, buses::Array{Bus,1})
    if height != 0
        # curr is not a leaf, so create ids for children
        # These ids double as indices for the arrays
        childrenids = (2*curr.id, 2*curr.id + 1)
        for i in childrenids
            # Create child, branch connecting child to curr
            buses[i] = genericPQbus(i)
            branches[i-1] = genericbranch(curr, buses[i])

            # Continue recursion on just-created child
            binarytreeaux!(height-1, buses[i], branches, buses)
        end
    end

    return nothing # function only used for side effects
end

# Returns a generic PQ bus with given ID, power consumption 1.0 W
function genericPQbus(id::Int)
    return Bus(id, -1.0, 1.0, -π/6, π/6, 0.0, 0.0, nothing, PQParams(1.0, 0.0))
end

# Creates a generic branch with given source / target, no impedance
function genericbranch(frombus::Bus, tobus::Bus)
    return Branch(frombus, tobus, 0.0, 0.0, 0.0, 0.0, 0.0)
end

In [ ]:
heights = 10
nrepetitions = 3

# Generic variables for later use
allres = 2
tol = 1.0
psp = PackSetupParams(fill(allres,4)...)

times = Array{Float64}(undef, heights)

for i = 1:heights
    network = createbinarytreenetwork(i)
    
    totaltime = @elapsed begin
        for j = 1:nrepetitions
            pwas = getpfpwas(network, psp, tol)
            # Perform final array multiplication, leaving P1 and Q1 unexposed
            finalpwa = clustertree(network, pwas, 1)
        end
    end
    
    times[i] = totaltime / nrepetitions # take average
end

println(times)

In [ ]:
buscounts = map(i -> 2^(i+1) - 1, 1:heights)
plt = plot(buscounts, times, scale=:log10)
plot!(plt, title = "Buses vs. Time to Solve", xlabel = "Number of buses", ylabel = "Average time", legend=:none)